# Stock Market Question Answering Agent

- Answer questions about individual stocks using (mostly) OpenBB functions
- Uses the OpenAI tools functionality, writing our own BB_agent_tool class
- Define a set of tool functions to call OpenBB
- When prompting, give OpenAI the set of tools and their descriptions in the prompt and also in the structured JSON format
- If OpenAI needs to run a tool function to respond to the prompt, it will provide the call signature and ask you to provide the return values before continuing

  

In [1]:
import os
import dotenv
import warnings
from datetime import datetime, timedelta

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

# class to make a tool and utility functions
from BB_agent_tool import BB_agent_tool, get_response, agent_query

import pdb

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
!pwd 

/Users/drucev/projects/AInewsbot


In [3]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')


pandas         2.2.2
obb            4.2.1
openai         1.30.1
langchain      0.2.0


# Connect to OpenBB

In [4]:
obb


OpenBB Platform v4.2.1

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /crypto
    /currency
    /derivatives
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /regulators

Extensions:
    - commodity@1.1.1
    - crypto@1.2.1
    - currency@1.2.1
    - derivatives@1.2.1
    - economy@1.2.1
    - equity@1.2.1
    - etf@1.2.1
    - fixedincome@1.2.1
    - index@1.2.1
    - news@1.2.1
    - regulators@1.2.1

    - benzinga@1.2.1
    - econdb@1.1.1
    - federal_reserve@1.2.1
    - fmp@1.2.1
    - fred@1.2.1
    - intrinio@1.2.1
    - oecd@1.2.1
    - polygon@1.2.1
    - sec@1.2.1
    - tiingo@1.2.1
    - tradingeconomics@1.2.1
    - yfinance@1.2.1    

In [5]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)


In [6]:
obb.user


UserSettings

id: 0669ba7a-5d5c-7b57-8000-8d790214c7f0
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'tiingo_token': SecretStr('**********'), 'polygon_api_key': SecretStr('**********'), 'intrinio_api_key': None, 'fred_api_key': SecretStr('**********'), 'tradingeconomics_api_key': None, 'fmp_api_key': SecretStr('**********'), 'econdb_api_key': None, 'benzinga_api_key': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot_open_export': False, 'plot_pywry_height': 762, 'plot_pywry_width': 1400, 'request_timeout': 60, 'show_warnings': False, 'table_s

In [7]:
obb.account


/account
    login
    logout
    save
    refresh
    

In [30]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']
# obb.user.credentials.benzinga_api_key = os.environ['BENZINGA_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()


KeyError: 'BENZINGA_API_KEY'

In [9]:
obb.news.company("GOOG",provider='yfinance')

OBBject

id: 0669ba7e-ebc0-780f-8000-7b32265bb4e8
results: [{'date': datetime.datetime(2024, 7, 20, 11, 47), 'title': '1 Top Artifici...
provider: yfinance
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'}, 's...

In [8]:
obb.equity.search("Merck", provider="sec").to_df().head(3)


,symbol,name,cik
0,MRK,"Merck & Co., Inc.",310158


In [9]:
symbol = "MRK"
company = "Merck"


In [10]:
obj = obb.equity.price.quote(symbol)


In [11]:
obj


OBBject

id: 0665362b-b84e-7560-8000-91d541c924ef
results: [{'symbol': 'MRK', 'asset_type': None, 'name': 'Merck & Co., Inc.', 'excha...
provider: fmp
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'fmp'}, 'standa...

In [12]:
obj.results


[FMPEquityQuoteData(symbol=MRK, asset_type=None, name=Merck & Co., Inc., exchange=NYSE, bid=None, bid_size=None, bid_exchange=None, ask=None, ask_size=None, ask_exchange=None, quote_conditions=None, quote_indicators=None, sales_conditions=None, sequence_number=None, market_center=None, participant_timestamp=None, trf_timestamp=None, sip_timestamp=None, last_price=129.49, last_tick=None, last_size=None, last_timestamp=2024-05-24 20:00:02+00:00, open=131.45, high=131.45, low=129.16, close=None, volume=5829244, exchange_volume=None, prev_close=131.09, change=-1.6, change_percent=-0.012204999999999999, year_high=133.1, year_low=99.14, price_avg50=127.7616, price_avg200=115.32135, avg_volume=8199988, market_cap=327973566900.0, shares_outstanding=2532810000, eps=0.9, pe=143.88, earnings_announcement=2024-07-30 12:30:00+00:00)]

In [13]:
obj.json()

'{"id":"0665362b-b84e-7560-8000-91d541c924ef","results":[{"symbol":"MRK","asset_type":null,"name":"Merck & Co., Inc.","exchange":"NYSE","bid":null,"bid_size":null,"bid_exchange":null,"ask":null,"ask_size":null,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":129.49,"last_tick":null,"last_size":null,"last_timestamp":"2024-05-24T20:00:02Z","open":131.45,"high":131.45,"low":129.16,"close":null,"volume":5829244,"exchange_volume":null,"prev_close":131.09,"change":-1.6,"change_percent":-0.012204999999999999,"year_high":133.1,"year_low":99.14,"price_avg50":127.7616,"price_avg200":115.32135,"avg_volume":8199988,"market_cap":327973566900.0,"shares_outstanding":2532810000,"eps":0.9,"pe":143.88,"earnings_announcement":"2024-07-30T12:30:00Z"}],"provider":"fmp","warnings":null,"chart":null,"extra":{"metadata":{"arguments":{"provid

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 129.35,
   'bid_size': 800,
   'ask': 129.49,
   'ask_size': 1800,
   'last_price': 129.49,
   'open': 131.45,
   'high': 131.45,
   'low': 129.16,
   'volume': 5946890,
   'prev_close': 131.09,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 127.9412,
   'ma_200d': 115.4383,
   'volume_average': 8213074.0,
   'volume_average_10d': 6263710.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'source': 'iex'}},
   'duration': 98372417,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-26T12:26:35.538728'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-26,112.0000,113.19,110.6501,111.07,6378339.0,111.3186,69766
2023-05-30,110.2222,110.34,109.0800,110.09,166016.0,109.8427,3155
2023-05-31,108.4700,111.15,108.4700,110.41,14563536.0,110.3172,91481
2023-06-01,110.6600,111.59,109.8800,110.93,6960063.0,110.9360,76972
2023-06-02,111.1100,112.73,110.7200,112.52,5792792.0,112.1935,74396
...,...,...,...,...,...,...,...
2024-05-20,130.8100,131.74,130.6500,130.99,4333726.0,131.2016,58819
2024-05-21,131.6800,132.79,130.4800,130.76,5280473.0,131.1038,66107
2024-05-22,130.1700,131.60,130.1500,131.10,4329193.0,131.1204,57070


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-26,420.02,420.7700,415.250,415.33,627251,93829975.0,418.9092
2023-05-30,420.18,422.5800,418.735,422.03,609041,72207960.0,420.4468
2023-05-31,417.85,419.2200,416.220,418.28,665147,110782530.0,418.0147
2023-06-01,421.82,422.9200,416.790,418.09,653406,88758018.0,420.4660
2023-06-02,427.92,428.7400,423.950,424.50,678309,91426171.0,426.8396
...,...,...,...,...,...,...,...
2024-05-20,530.06,531.5601,529.170,529.57,331748,37711090.0,530.2811
2024-05-21,531.36,531.5200,529.070,529.28,306611,33382000.0,530.4508
2024-05-22,529.83,531.3800,527.600,530.65,340278,47734744.0,529.7849


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='sec', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc.')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,56.56,2024-05-24 20:00:02+00:00,55.46,56.82,55.43,2344166,55.01,...,66.15,54.12,58.0400,59.83815,2951465,1.002300e+11,1772100000,4.36,12.97,2024-05-23 00:00:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,72.27,2024-05-24 20:00:01+00:00,72.64,73.09,72.17,6123549,72.34,...,79.49,48.66,73.3880,63.64275,7378571,1.284440e+11,1777280000,2.39,30.24,2024-07-16 12:30:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,200.71,2024-05-24 20:00:02+00:00,197.75,200.76,197.56,7218799,196.92,...,205.88,134.40,194.2518,167.96075,9108733,5.763729e+11,2871670000,16.56,12.12,2024-07-12 12:30:00+00:00
3,MS,Morgan Stanley,NYSE,98.97,2024-05-24 20:00:02+00:00,99.45,99.91,98.81,3547499,98.92,...,103.25,69.42,93.4398,86.17185,7814188,1.608421e+11,1625160000,5.50,17.99,2024-07-16 12:30:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [24]:
news_df = obb.news.company(symbol="LLY", provider='polygon', limit=10).to_df()

for item in news_df.itertuples():
    markdown_str = ""
    date_object = item.Index
    display_title = item.title.replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
    description = item.text.replace("$", "\\\$")
    markdown_str += f"[{str(date_object)} {display_title}]({item.url})\n {description}"
    display(Markdown(markdown_str))

[2024-06-03 13:45:00+00:00 Watch Out Wegovy: Here Comes Another Promising Anti-Obesity Candidate](https://www.fool.com/investing/2024/06/03/watch-out-wegovy-here-comes-another-promising-anti/)
 The fight for dominance in the weight loss market is just getting started.

[2024-06-03 19:14:17+00:00 Structure Therapeutics Obesity Candidate Is 'Very Competitive' To Eli Lilly's Orforglipron, Analyst Says](https://www.benzinga.com/analyst-ratings/analyst-color/24/06/39144875/structure-therapeutics-obesity-candidate-is-very-competitive-to-eli-lillys-orforgli)
 On Monday, Structure Therapeutics Inc. (NASDAQ:GPCR) revealed 12-week topline obesity data from its Phase 2a study of GSBR-1290.
In the Phase 2a obesity study, GSBR-1290 demonstrated a clinically meaningful and statistically significant placebo-adjusted mean decrease in weight of 6.2% at 12 weeks.
Cantor Fitzgerald maintains its bullish views on Structure Therapeutics and keeps the Overweight rating with a price target of \\$65.
The analyst says, “now have more conviction that ‘1290 is very competitive with Eli Lilly And Co’s (NYSE:LLY) orforglipron on efficacy and data from ...Full story available on Benzinga.com

[2024-06-04 15:45:00+00:00 ETFs to Profit from the Weight Loss Drug Boom](https://www.zacks.com/stock/news/2283455/etfs-to-profit-from-the-weight-loss-drug-boom)
 The market for GLP-1 drugs could soar to \\$130 billion by 2030

[2024-06-06 11:45:00+00:00 Is Viking Therapeutics Stock a Buy on the Dip?](https://www.fool.com/investing/2024/06/06/is-viking-therapeutics-stock-a-buy-on-the-dip/)
 Short-term thinking is creating an intriguing opportunity for investors.

[2024-06-06 14:21:00+00:00 Eli Lilly (LLY) Veteran Anat Ashkenazi to Join Alphabet as CFO](https://www.zacks.com/stock/news/2284718/eli-lilly-lly-veteran-anat-ashkenazi-to-join-alphabet-as-cfo)
 Eli Lilly (LLY) CFO Anat Ashkenazi joins Google's parent company Alphabet as CFO and senior vice president, effective from Jul 31, 2024

[2024-06-07 09:45:00+00:00 Is This Trend a Threat to Eli Lilly's Weight Loss Drug Dominance?](https://www.fool.com/investing/2024/06/07/is-this-trend-a-threat-to-eli-lillys-weight-loss-d/)
 The weight loss drug market is growing rapidly.

[2024-06-07 14:20:00+00:00 Pharma Stock Roundup: LLY's CFO Departs, ABBV, AZN Drugs Get CHMP Approval](https://www.zacks.com/stock/news/2285380/pharma-stock-roundup-llys-cfo-departs-abbv-azn-drugs-get-chmp-approval)
 Eli Lilly's (LLY) CFO Anat Ashkenazi resigns. CHMP recommends the expanded use of AbbVie's (ABBV) Skyrizi and AstraZeneca's (AZN) Tagrisso.

[2024-06-07 14:42:12+00:00 ETFs Bet High On Weight Loss Drugs, But Industry Observers Question Long-Term Potential](https://www.benzinga.com/analyst-ratings/analyst-color/24/06/39228957/etfs-bet-high-on-weight-loss-drugs-but-industry-observers-question-long-term-potent)
 With more than 10% of the global population now suffering from obesity, the share prices of companies providing the latest weight-loss drugs have surged.

However, according to industry observers, investors considering exposure through the newly launched weight-loss exchange-traded funds (ETFs) need to evaluate their potential.

The Roundhill GLP-1 Weight Loss ETF (NASDAQ:OZEM) and the Amplify Weight Loss Drug and Treatment ETF (NYSE:THNR) debuted on May 21 with an expense ratio of 0.59%.
Both ETFs have significant weightings in industry giants Eli Lilly And Co (NYSE:LLY) and Novo Nordisk A/S (NYSE:NVO), with OZEM and THNR allocating about 40% and 30%, respectively, to these companies.
Eli Lilly and ...Full story available on Benzinga.com

[2024-06-07 14:50:53+00:00 Wall Street Reacts To Hot Jobs Report: Stocks Swing As Traders Abandon July Rate Cut Hopes](https://www.benzinga.com/markets/equities/24/06/39229188/wall-street-reacts-to-hot-jobs-report-stocks-swing-as-traders-abandon-july-rate-cut-hopes)
 The May labor market report delivered hotter-than-expected numbers in terms of job gains and wage growth.
Last month, U.S. employers added 272,000 nonfarm payrolls, a sharp increase from April’s 165,000 and well above the expected 180,000.
Average hourly earnings witnessed a 4.1% year-on-year surge, up from an upwardly revised 4% in April, and above the expected 3.9%. On a monthly basis, earnings advanced at a 0.4% pace, accelerating from the previous 0.2%, and above the predicted 0.3%.
Overall, it was a hot jobs report on almost all fronts, except for a slight uptick in the unemployment rate from 3.9% to 4%.
The market interpreted these data as the final nail in the coffin for the already slim chances of a potential rate cut in July. Fed futures now indicate ...Full story available on Benzinga.com

[2024-06-08 14:05:00+00:00 The Company Behind Mounjaro and Zepbound Is Sponsoring Caitlin Clark. Should You Buy the Stock?](https://www.fool.com/investing/2024/06/08/the-company-behind-mounjaro-and-zepbound-is-sponso/)
 One of the most lucrative sports sponsorship opportunities is jersey patches, and one leading pharmaceutical company may have just struck gold.

In [25]:
news_df = obb.news.company(symbol="LLY", provider='benzinga', limit=10).to_df()

for item in news_df.itertuples():
    markdown_str = ""
    date_object = item.Index
    display_title = item.title.replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
    description = item.text.replace("$", "\\\$")
    markdown_str += f"[{str(date_object)} {display_title}]({item.url})\n {description}"
    display(Markdown(markdown_str))

OpenBBError: 
[Error] -> Missing credential 'benzinga_api_key'. Check https://www.benzinga.com to get it.

In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,tags,id,amp_url,publisher
date,,,,,,,,,,
2024-05-20 14:45:00+00:00,AstraZeneca (AZN) to Build $1.5B Cancer Drug P...,This new plant will be AstraZeneca's (AZN) fir...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2276201/astra...,"AZN,PFE,MRK",Zacks Equity Research,None,mRCY0S03pJh-0mZOQdgzBpD1tc09a63S4-xzHV1B9sw,https://www.zacks.com/amp/stock/news/2276201/a...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-22 13:50:11+00:00,Why Merck (MRK) is a Top Momentum Stock for th...,The Zacks Style Scores offers investors a way ...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2277274/why-m...,MRK,Zacks Equity Research,None,u-Pzf5KpgMBaRgI4tI6hnc8dCjVUV-gYNhCrshvOOpg,https://www.zacks.com/amp/stock/news/2277274/w...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-24 09:50:00+00:00,Billionaire Ken Griffin Is Buying These Divide...,The billionaire hedge fund manager doesn't nee...,[{'url': 'https://g.foolcdn.com/editorial/imag...,https://www.fool.com/investing/2024/05/24/bill...,"BAC,MRK,HES,CVX",newsfeedback@fool.com (Keith Speights),investing,bZDVTjsKEErP9Ogrz6RcUanUPRyov0URt8izBs3fs-0,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-24 15:03:44+00:00,Sell Alert: 2 Dividend Stocks To Sell In May A...,September is the consistent worst month for th...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4695496-sell-...,"ABBV,AMGN,BAH,BMY,DKS,IBM,PFE,XOM,WSM,MRK",Robert & Sam Kovacs,None,kZi_Z2OKRAq2j3OfMEDWO6i2i4w8OWSVArwDG6pFtYs,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-24 17:41:19+00:00,Why Is Cancer-Focused Merus Stock Trading Over...,"Thursday, Merus N.V. (NASDAQ:MRUS) announced t...",[{'url': 'https://cdn.benzinga.com/files/image...,https://www.benzinga.com/analyst-ratings/analy...,"MRK,MRUS",Vandana Singh,"Analyst Color,Biotech,Health Care,Analyst Rati...",-YDejnW7jjgd3AiAI2lljjroXf_d6b3CFyZ5L7svJbs,https://www.benzinga.com/amp/content/39012157,{'favicon_url': 'https://s3.polygon.io/public/...


# Prompt OpenAI 

In [21]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0

client = OpenAI()


In [22]:
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(client, messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The question of the airspeed velocity of an unladen swallow is often associated with a famous line from the movie "Monty Python and the Holy Grail." In a more scientific context, it is known that the airspeed velocity of a European Swallow (Hirundo rustica) is approximately 11 meters per second or around 24 miles per hour, based on studies of their flight patterns and speeds.

However, it's important to note that this measurement can vary depending on factors such as wind conditions, the bird's health, and its specific mode of flight (e.g., cruising versus diving).


# Question Answering Agent

In [23]:
# make a custom tool

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "default_parameters": {},
    "example_parameter_values": [{
        "symbol": "MSFT",
    }],
}

tool = BB_agent_tool(**fn_metadata)
tool_response = tool(symbol="MSFT")
len(json.loads(tool_response)[0]['item1'])


143226

In [24]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(client, user_message, verbose=True)



Role: You are an AI stock market assistant tasked with providing investors
with up-to-date, detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate,
complete, but concise information relevant to their questions about individual
stocks.

Capabilities: You are given a number of tools as functions. Use as many tools
as needed to ensure all information provided is timely, accurate, concise,
relevant, and responsive to the user's query.

Instructions:
1. Input validation. Determine if the input is asking about a specific company
or stock ticker. If not, respond in a friendly, positive, professional tone
that you don't have information to answer and suggest alternative services
or approaches.

2. Symbol extraction. If the query is valid, extract the company name or ticker
symbol from the question. If a company name is given, look up the ticker symbol
using a tool. If the ticker symbol is not found based on the company, try to
corre

**Summary of Item 1 from Microsoft's Latest Annual Report (Form 10-K):**

Microsoft's latest Annual Report highlights its mission and core business focus areas:

1. **General Business Overview**:
   - Microsoft is a technology company aiming to empower every individual and organization globally.
   - The company creates platforms and tools powered by AI to enhance business competitiveness, improve educational and health outcomes, and grow public-sector efficiency.

2. **Products and Services**:
   - Microsoft offers an array of services, including cloud solutions (Azure), software products (Windows, Office 365), development tools, and devices (Surface, Xbox).
   - Key business processes are driven by AI and cloud computing capabilities.
   - The company emphasizes responsible AI, aligning with principles of fairness, reliability, privacy, security, inclusiveness, transparency, and accountability.

3. **Strategic Ambitions**:
   - **Reinvent Productivity and Business Processes**: Through products like Microsoft 365, Dynamics 365, and LinkedIn.
   - **Build the Intelligent Cloud and Intelligent Edge Platform**: Focus on Azure cloud services, AI platforms, and hybrid infrastructure.
   - **Create More Personal Computing**: Development of Windows 11, AI-backed applications, and new gaming experiences.

4. **Corporate Priorities**:
   - Commitment to sustainability, targeting carbon negativity, water positivity, and zero waste by 2030.
   - Addressing racial injustice through specific initiatives focused on strengthening communities, engaging ecosystems, and increasing representation within the company.
   - Investing in digital skills training and AI education for global workers.

5. **Human Capital Resources**:
   - Focus on attracting diverse talent and fostering an inclusive work environment.
   - Various employee support programs including wellness, flexible work options, and continuous learning opportunities.
   - Emphasis on pay equity and transparency in workforce diversity data.

6. **Operational Segments**:
   - **Productivity and Business Processes**: Includes Office, LinkedIn, and Dynamics business solutions.
   - **Intelligent Cloud**: Encompasses Azure, server products, AI services, and enterprise services.
   - **More Personal Computing**: Involves Windows, devices (Surface, HoloLens), gaming (Xbox), and search and news advertising (Bing, Microsoft Edge).

7. **Commitment to Innovation**:
   - Investment in research and development across a broad range of technologies to drive future growth.
   - Focus on integrating AI across products and services for enhanced customer experiences.

8. **Corporate Social Responsibility**:
   - Detailed on environmental sustainability efforts, racial justice initiatives, and commitments to digital inclusion and skills development.

This summary reflects the essence of Microsoft's strategic vision, core products, and its commitment to innovation, sustainability, and social equity.

# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [25]:
# creates examples by calling the tool on the example parameters and updates descriptions
openai_tools=[]
tool_descs=""

for v in BB_agent_tool.agent_registry.values():
        t = v.tooldict
        openai_tools.append(t)
        tname = t['function']['name']
        tdesc = t['function']['description']
        tool_descs += f"{tname} : {tdesc}"
        if v.example_code:
            tool_descs += f" Usage: {v.example_code}"
        tool_descs += "\n---\n"

tool_descs

'get_10k_item1_from_symbol : Given a stock symbol, gets item 1 of the company\'s latest 10-K annual report filing. Usage: get_10k_item1_from_symbol(symbol="MSFT") -> [{"item1": "Note About Forward-Looking Statements\\nThis report includes estimates, projections, statements relating to our business plans, objectives, and expected operating results that are \\u201cforward-looking statements\\u201d within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. Forward-looking statements may appear throughout this report, including the following sections: \\u201cBusiness\\u201d (Part I, Item 1 of this Form 10-K), \\u201cRisk Factors\\u201d (Part I, Item 1A of this Form 10-K), and \\u201cManagement\\u2019s Discussion and Analysis of Financial Condition and Results of Operations\\u201d (Part II, Item 7 of this Form 10-K). These forward-looking statements generally are identified by

In [26]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]


"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [27]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [28]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [29]:
fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            },""
        "limit": {
            "type": "integer",
            "description": "The number of results to return. Pick a small number from 1 to 10 and choose the best response."
            }
        },
    "default_parameters": {
        "provider": "sec"
    },
    "example_parameter_values": [{
        "query": "Broadcom",
    }],
    # "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)



In [30]:
BB_agent_tool.agent_registry['get_equity_search_symbol'](query="Southwest Air")

'[{"symbol": "LUV", "name": "SOUTHWEST AIRLINES CO", "cik": "92380"}]'

In [31]:
agent_query(client, "What is the stock symbol for Southwest airlines?", verbose=False)


get_equity_search_symbol({'query': 'Southwest Airlines', 'limit': 1}) -> [{"symbol": "LUV", "name": "SOUTHWEST AIRLINES CO", "cik": "92380"}]


The stock symbol for Southwest Airlines is **LUV**.

In [32]:
agent_query(client, "What is the stock symbol for yakimee air?", verbose=False)


get_equity_search_symbol({'query': 'yakimee air', 'limit': 10}) -> []


I'm sorry, but I couldn't find any stock symbol for "yakimee air." Could you please confirm the company name or provide more details? It's possible that there might have been a slight misspelling or the company might be listed under a different name.

In [33]:
# agent is able to first get symbol, then get annual report for symbol
agent_query(client, "What is item 1 from the 10k annual report for Microsfot?", verbose=True)



Role: You are an AI stock market assistant tasked with providing investors
with up-to-date, detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate,
complete, but concise information relevant to their questions about individual
stocks.

Capabilities: You are given a number of tools as functions. Use as many tools
as needed to ensure all information provided is timely, accurate, concise,
relevant, and responsive to the user's query.

Instructions:
1. Input validation. Determine if the input is asking about a specific company
or stock ticker. If not, respond in a friendly, positive, professional tone
that you don't have information to answer and suggest alternative services
or approaches.

2. Symbol extraction. If the query is valid, extract the company name or ticker
symbol from the question. If a company name is given, look up the ticker symbol
using a tool. If the ticker symbol is not found based on the company, try to
corre

**Item 1 from the 10-K Annual Report for Microsoft (Symbol: MSFT)**

**General Overview**:
Microsoft is a technology company aiming to empower individuals and organizations globally to achieve more. They focus on creating local opportunities, growth, and impactful outcomes worldwide, leveraging platforms and tools powered by artificial intelligence (AI) to deliver superior solutions to support businesses, educational and health outcomes, improve public sector efficiency, and foster human ingenuity.

**Key Highlights**:

1. **AI Integration**:
   - Microsoft is innovating and expanding its portfolio with AI capabilities to help organizations overcome challenges and emerge stronger.
   - Their investment in AI spans the entire company, from Microsoft Teams and Outlook to Bing and Xbox, infusing generative AI capabilities across services.

2. **Core Principles for AI**:
   - Microsoft's AI work is guided by principles including fairness, reliability and safety, privacy and security, inclusiveness, transparency, and accountability.

3. **Products and Services**:
   - Microsoft offers a myriad of services, including cloud-based solutions, operating systems, productivity applications, business solutions, software development tools, and devices like PCs, tablets, and gaming consoles.

4. **Business Segments**:
   - Key ambitions include reinventing productivity and business processes, building the intelligent cloud and intelligent edge platform, and creating more personal computing.

5. **Corporate Social Responsibility**:
   - Commitment to Sustainability: Microsoft aims to be carbon negative, water positive, and zero waste by 2030.
   - Addressing Racial Injustice and Inequity: Microsoft is working to improve the lives of Black and African American communities in the U.S. and beyond.

6. **Human Capital Resources**:
   - Microsoft aims to recruit, develop, and retain a diverse range of talent, creating an inclusive and supportive work environment for their 221,000 employees globally.

Microsoft’s report details significant investment in AI, strong ethical guidelines, and emphasizes corporate social responsibility and sustainability.

For more in-depth details, please refer to Microsoft’s full 10-K report.

Feel free to ask if you need more specific information or have further queries!

In [34]:
fn_metadata = {
    "name": "get_equity_price_quote",
    "description": "Given a stock symbol, get latest market data including last price in JSON format.",
    "openapi_path" : '/api/v1/equity/price/quote',
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol to get quote data for."
            },
        },
    "default_parameters": {
        "provider": "yfinance"
    },
    "example_parameter_values": [{
        "symbol": "AVGO",
    }],
    # "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)

In [35]:
agent_query(client, f"What is the last market quote for symbol {symbol}?", verbose=False)


get_equity_price_quote({'symbol': 'MRK'}) -> [{"symbol": "MRK", "asset_type": "EQUITY", "name": "Merck & Co., Inc.", "exchange": "NYQ", "bid": 12...


The last market quote for Merck & Co., Inc. (symbol: MRK) is as follows:

- **Last Price**: \\$129.49
- **Bid**: \\$129.35 (800 shares)
- **Ask**: \\$129.49 (1800 shares)
- **Open**: \\$131.45
- **High**: \\$131.45
- **Low**: \\$129.16
- **Volume**: 5,946,890
- **Previous Close**: \\$131.09
- **52-week High**: \\$133.10
- **52-week Low**: \\$99.14
- **50-day Moving Average**: \\$127.94
- **200-day Moving Average**: \\$115.44

All prices are in USD.

In [36]:
agent_query(client, f"What is the last market quote for Consolidated Agglomerators", verbose=False)


get_equity_search_symbol({'query': 'Consolidated Agglomerators', 'limit': 5}) -> []


It seems I couldn't find a stock symbol for "Consolidated Agglomerators." Could you please check the company name for any spelling errors or provide more details?

In [37]:
agent_query(client, f"What is the current weather in Las Vegas", verbose=False)


I’m sorry, but I don't have information about the weather. For the current weather in Las Vegas, you can check a weather forecasting website or app like Weather.com, AccuWeather, or your local news station.

In [38]:
tool = BB_agent_tool(
    name="get_company_profile_json",
    description="Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    openapi_path='/api/v1/equity/profile',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"Can you provide a basic company profile of symbol {symbol}", verbose=False)


get_company_profile_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "0000310158", "cusip": "58933Y105", "isin": "...


Here is a basic company profile for Merck & Co., Inc. (symbol: MRK):

- **Name:** Merck & Co., Inc.
- **Stock Exchange:** New York Stock Exchange
- **Sector:** Healthcare
- **Industry Category:** Drug Manufacturers - General
- **Headquarters:** 
  - Address: 2000 Galloping Hill Road, Kenilworth, NJ, 07033, United States
  - Phone: 908 740 4000
- **CEO:** Mr. Robert M. Davis J.D.
- **Employees:** 70,000
- **Website:** [merck.com](https://www.merck.com)
- **Description:**
  Merck & Co., Inc. operates as a healthcare company worldwide. It has two segments: Pharmaceutical and Animal Health. The Pharmaceutical segment offers human health pharmaceutical products in areas such as oncology, hospital acute care, immunology, neuroscience, virology, cardiovascular, and diabetes, as well as vaccines. The Animal Health segment deals with veterinary pharmaceuticals and digitally connected identification products. The company collaborates with other major pharmaceuticals to jointly develop and commercialize treatments. Merck & Co., Inc. was founded in 1891 and is headquartered in Kenilworth, New Jersey.
- **Stock Information:**
  - **Market Cap:** \\$327.97 billion
  - **Last Price:** \\$129.49
  - **52-Week High:** \\$133.1
  - **52-Week Low:** \\$99.14
  - **Average Volume:** 8,199,988
  - **Annualized Dividend:** \\$3.08
  - **Beta:** 0.402

![Merck Logo](https://financialmodelingprep.com/image-stock/MRK.png)

If you need more detailed information or have any other questions, feel free to ask!

In [39]:
# tool = BB_agent_tool(
#     name="get_equity_shorts_short_interest",
#     description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
#     openapi_path='/api/v1/equity/shorts/short_interest',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     default_parameters={
#     },    
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# agent_query(client, f"Provide latest statistics on short interest for symbol {symbol}", verbose=False)


In [40]:
tool = BB_agent_tool(
    name="get_equity_fundamental_historical_splits",
    description="Given a stock symbol, get the company's historical stock splits in JSON format.",
    openapi_path='/api/v1/equity/fundamental/historical_splits',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [41]:
agent_query(client, f"Provide historical split information for symbol AAPL", verbose=False)

get_equity_fundamental_historical_splits({'symbol': 'AAPL'}) -> [{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August ...


Here are the historical stock splits for Apple Inc. (symbol: AAPL):

1. **August 31, 2020**: 4-for-1
2. **June 9, 2014**: 7-for-1
3. **February 28, 2005**: 2-for-1
4. **June 21, 2000**: 2-for-1
5. **June 16, 1987**: 2-for-1

If you need more information, feel free to ask!

In [42]:
tool = BB_agent_tool(
    name="get_balance_sheet_json",
    description="Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/balance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    default_parameters={
    },
    singular=1
)

agent_query(client, f"what are the latest total assets for symbol {symbol}", verbose=False)


get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The latest total assets for Merck & Co., Inc. (symbol: MRK) as of December 31, 2023, are \\$106,675,000,000 USD.

For more details, you can refer to their [latest filing with the SEC](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [43]:
# can give by year
# obb.equity.fundamental.balance(symbol='NVDA', period='annual', fiscal_year='2023', limit=1)
# also growth from prior period obb.equity.fundamental.balance_growth

In [44]:
tool = BB_agent_tool(
    name="get_cash_flow_json",
    description="Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/cash',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent cash flow from operations for stock symbol {symbol}", verbose=False)


get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent cash flow from operations for Merck & Co. Inc. (symbol: MRK) is \\$13,006,000,000 for the fiscal year ending on December 31, 2023.

For further details, you can refer to their [10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [45]:
tool = BB_agent_tool(
    name="get_income_statement_json",
    description="Given a stock symbol, get the latest income statement data for the company in JSON format",
    openapi_path='/api/v1/equity/fundamental/income',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent net income for symbol {symbol}", verbose=False)


get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent net income for Merck & Co., Inc. (symbol: MRK) for the fiscal year ending on December 31, 2023, was \\$365,000,000.

For more details, you can access [their filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [46]:
tool = BB_agent_tool(
    name="get_fundamental_metrics_json",
    description="Given a stock symbol, get fundamental metrics for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/metrics',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the most recent PE ratio for stock symbol {symbol}", verbose=False)


get_equity_price_quote({'symbol': 'MRK'}) -> [{"symbol": "MRK", "asset_type": "EQUITY", "name": "Merck & Co., Inc.", "exchange": "NYQ", "bid": 12...
get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 276583740000.0, "pe_ratio": 757.7636712328766, "period_ending": "20...


The most recent PE ratio for Merck & Co., Inc. (symbol: MRK) is approximately 757.76, as of the fiscal year ending December 31, 2023.

In [47]:
tool = BB_agent_tool(
    name="get_fundamental_ratios_json",
    description="Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/ratios',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    default_parameters={
    },
    singular=1
)

agent_query(client, f"what was the most recent price to sales ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 276583740000.0, "pe_ratio": 757.7636712328766, "period_ending": "20...


The most recent price-to-sales ratio for Merck & Co., Inc. (symbol: MRK) is 4.62.

In [48]:
tool = BB_agent_tool(
    name="get_equity_fundamental_multiples",
    description="Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/multiples',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent revenue per share for stock symbol {symbol}", verbose=False)


get_equity_fundamental_multiples({'symbol': 'MRK'}) -> {"symbol": "MRK", "revenue_per_share_ttm": 24.25424397947098, "net_income_per_share_ttm": 0.91038294...


The most recent revenue per share for Merck & Co., Inc. (ticker symbol: MRK) is \\$24.25 (TTM: Trailing Twelve Months).

In [49]:
# tool = BB_agent_tool(
#     name="get_historical_eps",
#     description="Given a stock symbol, get historical earnings per share data for the company in JSON format.",
#     openapi_path='/api/v1/equity/fundamental/historical_eps',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

#

In [50]:
# agent_query(client, f"what was the EPS for {company} in the quarter ended 2022-09-30?", verbose=False)


In [51]:
tool = BB_agent_tool(
    name="get_equity_fundamental_dividend",
    description="Given a stock symbol, get the latest dividend data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/dividends',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the latest dividend data for stock symbol {symbol}?", verbose=False)


get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


The latest dividend data for Merck & Co., Inc. (symbol: MRK) is as follows:

- **Ex-Dividend Date:** March 14, 2024
- **Dividend Amount:** \\$0.77
- **Record Date:** March 15, 2024
- **Payment Date:** April 5, 2024
- **Declaration Date:** January 23, 2024

If you need more information, feel free to ask!

In [52]:
agent_query(client, f"what was the dividend data as of the end of 2023 for stock symbol {symbol}?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}, {"symbol": "NMRK", "name": "NEWMAR...
get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


As of the end of 2023, Merck & Co., Inc. (symbol: MRK) had the following dividend data:

- **Ex-Dividend Date:** December 14, 2023
- **Dividend Amount:** \\$0.77 per share
- **Record Date:** December 15, 2023
- **Payment Date:** January 8, 2024
- **Declaration Date:** November 28, 2023

For reference, earlier in 2023, the dividends were as follows:
- **Ex-Dividend Date:** September 14, 2023, Dividend Amount: \\$0.73
- **Ex-Dividend Date:** June 14, 2023, Dividend Amount: \\$0.73
- **Ex-Dividend Date:** March 14, 2023, Dividend Amount: \\$0.73

In [53]:
tool = BB_agent_tool(
    name="get_trailing_dividend_yield_json",
    description="Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    openapi_path='/api/v1/equity/fundamental/trailing_dividend_yield',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}, {"symbol": "NMRK", "name": "NEWMAR...
get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{"date": "2023-05-25", "trailing_dividend_yield": 0.0252894034}, {"date": "2023-05-26", "trailing_d...


The trailing dividend yield for Merck & Co., Inc. (symbol: MRK) at the end of 2023 was approximately 2.71%.

In [54]:
tool = BB_agent_tool(
    name="get_price_performance_json",
    description="Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    openapi_path='/api/v1/equity/price/performance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the performance for {company} from 1 year ago?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}]
get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": -0.012204999999999999, "wtd": null, "one_week": -0.010091000000000001,...


The performance for Merck & Co., Inc. (symbol: MRK) over the past year is a gain of approximately 16.58%.

In [55]:
# this is unreliable, chatgpt can't always find the higest value in a large dict
tool = BB_agent_tool(
    name="get_etf_equity_exposure_json",
    description="Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    openapi_path='/api/v1/etf/equity_exposure',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [56]:
pd.DataFrame(json.loads(tool(symbol='MRK'))).sort_values('weight', ascending=False)

,equity_symbol,etf_symbol,shares,weight,market_value
437,MRK,FTXH,11684.0,0.075700,1.512961e+06
84,MRK,IUHC.L,1132285.0,0.061898,1.484312e+08
13,MRK,XLV,18406426.0,0.061826,2.383448e+09
81,MRK,QDVG.DE,1134961.0,0.061364,1.488955e+08
83,MRK,IUHE.AS,0.0,0.061238,1.484426e+08
...,...,...,...,...,...
665,MRK,GPAL,640.0,NaN,6.631040e+04
677,MRK,GDEF,397.0,NaN,5.204273e+04
685,MRK,EUNU.DE,50000.0,NaN,3.868500e+04
698,MRK,GSEU,121.0,NaN,2.214566e+04


In [57]:
agent_query(client, f"which ETF has the highest weight in {company} ?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}]
get_etf_equity_exposure_json({'symbol': 'MRK'}) -> [{"equity_symbol": "MRK", "etf_symbol": "VTSAX", "shares": 79744771.0, "weight": 0.0067, "market_val...


The ETF with the highest weight in Merck (symbol: MRK) is the **Health Care Select Sector SPDR Fund (symbol: XLV)**, with a weight of **0.06182614**.

In [58]:
# make a custom tool
# really might as well write a function to return top 10 etfs with highest weight in company

# def get_largest_from_json(json_str, field_name):
#     d = json.loads(json_str)
#     df = pd.DataFrame(d).sort_values(field_name, ascending=False)
#     return [{'largest': df.iloc[0].to_json()}]


# tool = BB_agent_tool(
#     name="get_largest_from_json",
#     description="Given a json string json_str representing a list of objects, and a field name field_name, return the object with the largest value for field_name.",
#     openapi_path=None,
#     callable=get_largest_from_json,
#     parameters={
#         "json_str": {
#             "type": "string",
#             "description": "a JSON string representing a list of objects"
#             },
#         "field_name": {
#             "type": "string",
#             "description": "the field to search for the largest value"
#             },
#         },
#     default_parameters={
#     },
#     example_parameter_values=[{
#         "json_str": '[{"name": "low", "value": 0}, {"name": "high", "value": 1}]',
#         "field_name": 'value'
#     }],
# )

# tool_response = tool(json_str='[{"name": "low", "value": 0}, {"name": "high", "value": 1}]',
#                     field_name='value')
# tool_response
# len(json.loads(tool_response)[0]['item1'])


In [59]:
[(k, t.description) for k, t in BB_agent_tool.agent_registry.items()]

[('get_10k_item1_from_symbol',
  "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing."),
 ('get_equity_search_symbol',
  'Given a search string, get the stock symbol of the top company whose name best matches the search string.'),
 ('get_equity_price_quote',
  'Given a stock symbol, get latest market data including last price in JSON format.'),
 ('get_company_profile_json',
  'Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format'),
 ('get_equity_fundamental_historical_splits',
  "Given a stock symbol, get the company's historical stock splits in JSON format."),
 ('get_balance_sheet_json',
  'Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.'),
 ('get_cash_flow_json',
  'Given a stock symbol, get the latest cash flow statement data for the company in JSON format.'),
 ('get_income_statement_json',
  'Given a stock

In [60]:
# for k, t in BB_agent_tool.agent_registry.items():
#     print(t.example_code)

In [61]:
len(" ".join([(t.example_code) for k, t in BB_agent_tool.agent_registry.items()]))

23792